<span style="color:red"> **Warning!** </span> Runtime of approx 15 minutes 

In [1]:
## import libraries

import pandas as pd
import glob
from tqdm import tqdm_notebook

from pathlib import Path
import sys

In [2]:
## import custom functions

# Create path to custom functions
funcs_path = Path.cwd().parents[1]
sys.path.insert(1, str(funcs_path))

from funcs import paths

In [3]:
## Get path to temp electricity data

path_to_SM_electricity_temp_data_folder = paths.get_path_to_temp_data("elec_data")

path_to_SM_electricity_temp_data_folder

'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\temp\\elec_data'

In [13]:
%%time

raw_elec_data = pd.read_pickle(
    path_to_SM_electricity_temp_data_folder + "\raw_Elec_data.pkl", compression="xz"
)

raw_elec_data.head()

Wall time: 45.3 s


ID    Day  Hour
1000  195  1       0.117004
           2       0.049988
           3       0.024994
           4       0.024994
           5       0.049011
Name: Elec, dtype: float16

In [15]:
%%time

raw_elec_data = raw_elec_data.to_frame()
raw_elec_data.reset_index(inplace=True)

raw_elec_data.head()

Wall time: 9.26 s


,ID,Day,Hour,Elec
0,1000,195,1,0.117004
1,1000,195,2,0.049988
2,1000,195,3,0.024994
3,1000,195,4,0.024994
4,1000,195,5,0.049011


In [16]:
%%time

# Add Year column to df

# @numba.vectorize
def add_year (Day):
    
    if Day <= 365:
        return 2009
    elif Day <= 730:
        return 2010
    elif Day <= 1095:
        return 2011
    
add_year = np.vectorize(add_year)        

raw_elec_data["Year"] = add_year(raw_elec_data["Day"].values)

Wall time: 36.6 s


In [17]:
%%time

# Convert days to within 365

def convert_days (Day):
    
    if Day <= 365:
        return Day
    else:
        return Day%365

convert_days = np.vectorize(convert_days) 

raw_elec_data["Day"] = convert_days(raw_elec_data["Day"].values)

Wall time: 37.7 s


In [18]:
%%time

# Convert Year & Day to dates

def compose_date(
    years,
    months=1,
    days=1,
    weeks=None,
    hours=None,
    minutes=None,
    seconds=None,
    milliseconds=None,
    microseconds=None,
    nanoseconds=None,
):
    """ Docstring:
        
        Source: https://stackoverflow.com/questions/34258892/converting-year-and-day-of-year-into-datetime-index-in-pandas
        
        """
    
    years = np.asarray(years) - 1970
    months = np.asarray(months) - 1
    days = np.asarray(days) - 1
    types = (
        "<M8[Y]",
        "<m8[M]",
        "<m8[D]",
        "<m8[W]",
        "<m8[h]",
        "<m8[m]",
        "<m8[s]",
        "<m8[ms]",
        "<m8[us]",
        "<m8[ns]",
    )
    vals = (
        years,
        months,
        days,
        weeks,
        hours,
        minutes,
        seconds,
        milliseconds,
        microseconds,
        nanoseconds,
    )
    return sum(np.asarray(v, dtype=t) for t, v in zip(types, vals) if v is not None)

raw_elec_data["Date"] = compose_date(raw_elec_data['Year'], days=raw_elec_data['Day'])

Wall time: 16.4 s


In [20]:
%%time

raw_elec_data.drop(columns=["Day","Year"],inplace=True)

raw_elec_data["Date"] = pd.to_datetime(raw_elec_data["Date"])
raw_elec_data.set_index(["ID","Date","Hour"],inplace=True)

In [21]:
# convert to series
raw_elec_data = raw_elec_data["Elec"]

print("Reformatted Series with Dates")
raw_elec_data.head()

Reformatted Series with Dates


ID    Date        Hour
1000  2009-07-14  1       0.117004
                  2       0.049988
                  3       0.024994
                  4       0.024994
                  5       0.049011
Name: Elec, dtype: float16

In [24]:
%%time

raw_elec_data.to_pickle(
    path_to_SM_electricity_temp_data_folder + "\half_hourly_elec_data.pkl.pkl",
    compression="xz",
)

In [ ]:
# os.remove(r"temp\raw_elec_data.pkl")